In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("drive/MyDrive/wiki_movie_plots_deduped.csv")

In [ ]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [ ]:
df.head(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [ ]:
df["Plot"].head(5)

0    A bartender is working at a saloon, serving dr...
1    The moon, painted with a smiling face hangs ov...
2    The film, just over a minute long, is composed...
3    Lasting just 61 seconds and consisting of two ...
4    The earliest known adaptation of the classic f...
Name: Plot, dtype: object

In [ ]:
df.drop(["Release Year","Origin/Ethnicity","Director","Cast","Genre","Wiki Page"],axis=1,inplace=True)
df.head(5)

,Title,Plot
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr..."
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov..."
2,The Martyred Presidents,"The film, just over a minute long, is composed..."
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...


## Preprocessing of raw data

In [ ]:
#Lowercasing
df["Plot"]=df["Plot"].str.lower()
df.head(5)

,Title,Plot
0,Kansas Saloon Smashers,"a bartender is working at a saloon, serving dr..."
1,Love by the Light of the Moon,"the moon, painted with a smiling face hangs ov..."
2,The Martyred Presidents,"the film, just over a minute long, is composed..."
3,"Terrible Teddy, the Grizzly King",lasting just 61 seconds and consisting of two ...
4,Jack and the Beanstalk,the earliest known adaptation of the classic f...


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
df["tokens"] = list(map(tokenizer.tokenize,df["Plot"]))
df.head()

,Title,Plot,tokens
0,Kansas Saloon Smashers,"a bartender is working at a saloon, serving dr...","[a, bartender, is, working, at, a, saloon, ser..."
1,Love by the Light of the Moon,"the moon, painted with a smiling face hangs ov...","[the, moon, painted, with, a, smiling, face, h..."
2,The Martyred Presidents,"the film, just over a minute long, is composed...","[the, film, just, over, a, minute, long, is, c..."
3,"Terrible Teddy, the Grizzly King",lasting just 61 seconds and consisting of two ...,"[lasting, just, 61, seconds, and, consisting, ..."
4,Jack and the Beanstalk,the earliest known adaptation of the classic f...,"[the, earliest, known, adaptation, of, the, cl..."


In [ ]:
#Removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")
def remove_stopwords(text):
    res = []
    for token in text:
        if token not in english_stopwords:
            res.append(token)
    return res
df["tokens"] = list(map(remove_stopwords,df["tokens"]))
df.head(5)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Title,Plot,tokens
0,Kansas Saloon Smashers,"a bartender is working at a saloon, serving dr...","[bartender, working, saloon, serving, drinks, ..."
1,Love by the Light of the Moon,"the moon, painted with a smiling face hangs ov...","[moon, painted, smiling, face, hangs, park, ni..."
2,The Martyred Presidents,"the film, just over a minute long, is composed...","[film, minute, long, composed, two, shots, fir..."
3,"Terrible Teddy, the Grizzly King",lasting just 61 seconds and consisting of two ...,"[lasting, 61, seconds, consisting, two, shots,..."
4,Jack and the Beanstalk,the earliest known adaptation of the classic f...,"[earliest, known, adaptation, classic, fairyta..."


## Creation of inverted index

In [ ]:
def inv_ind_create(token_df):
    inverted_index = dict()
    for doc_id in range(len(token_df)):
        for term in token_df.iloc[doc_id]:
            if term not in inverted_index:
                inverted_index[term] = list()
            inverted_index[term].append(doc_id)
    for key in inverted_index:
        inverted_index[key] = list(set(inverted_index[key]))
    return inverted_index

In [ ]:
inverted_index = inv_ind_create(df["tokens"])
print(inverted_index["bartender"])

[0, 16385, 12290, 11781, 23046, 15879, 522, 12812, 4624, 7184, 11280, 16918, 1047, 13337, 12319, 4642, 4134, 16423, 9769, 13865, 16938, 26165, 15929, 10813, 8766, 7231, 21569, 26690, 8775, 7759, 13391, 14927, 9298, 8790, 11350, 13398, 12381, 11870, 7271, 13928, 5737, 10859, 5229, 2159, 6767, 11887, 13425, 8307, 14966, 2685, 2178, 13444, 23687, 6797, 13454, 17046, 8858, 21148, 5792, 20646, 13482, 10924, 20652, 14000, 8371, 17077, 9916, 192, 16580, 15045, 15559, 10447, 11477, 13016, 22233, 16605, 9951, 8928, 14050, 6884, 231, 13032, 1772, 6380, 15598, 34541, 16626, 20723, 14070, 16631, 14072, 12025, 13050, 13568, 13058, 3331, 2820, 21766, 16144, 9492, 16668, 13087, 15657, 13103, 3891, 34099, 5433, 22330, 13633, 10569, 1357, 9556, 1371, 11614, 21346, 3428, 9060, 14692, 13673, 15215, 11122, 17267, 13686, 12151, 13182, 16258, 13699, 11652, 21378, 23432, 13197, 11662, 13204, 16789, 21398, 7069, 10656, 5026, 2468, 21414, 21934, 9142, 6583, 10678, 14782, 3009, 14273, 10180, 11717, 4555, 6603, 

## Stemming

In [ ]:
from nltk.stem import PorterStemmer

def corpus_stemming(token_df):
    porter = PorterStemmer()
    document_stemmer = lambda doc: [porter.stem(word) for word in doc]
    return list(map(document_stemmer,token_df))

stem_token = corpus_stemming(df["tokens"])
stem_token[0]

['bartend',
 'work',
 'saloon',
 'serv',
 'drink',
 'custom',
 'fill',
 'stereotyp',
 'irish',
 'man',
 'bucket',
 'beer',
 'carri',
 'nation',
 'follow',
 'burst',
 'insid',
 'assault',
 'irish',
 'man',
 'pull',
 'hat',
 'eye',
 'dump',
 'beer',
 'head',
 'group',
 'begin',
 'wreck',
 'bar',
 'smash',
 'fixtur',
 'mirror',
 'break',
 'cash',
 'regist',
 'bartend',
 'spray',
 'seltzer',
 'water',
 'nation',
 'face',
 'group',
 'policemen',
 'appear',
 'order',
 'everybodi',
 'leav',
 '1']

In [ ]:
def query_parse(tokens):
    precedence = dict()
    precedence['('] = 0
    precedence[')'] = 0
    precedence['OR'] = 1
    precedence['AND'] = 2
    precedence['NOT'] = 3
    
    output = []
    op_stack = []
    
    for token in tokens:
        if(token == '('):
            op_stack.append(token)
        elif(token == ')'):
            operator = op_stack.pop()
            while(operator != '('):
                output.append(operator)
                operator = op_stack.pop()
        elif(token in precedence):
            if(op_stack):
                curr_operator = op_stack[-1]
                while(op_stack and precedence[curr_operator]>precedence[token]):
                    output.append(op_stack.pop())
                    if(op_stack):
                        curr_operator = op_stack[-1]
            op_stack.append(token)
        else:
            output.append(token.lower())
    while(op_stack):
        output.append(op_stack.pop())
    return output

## Boolean Retrieval

In [ ]:
def boolean_query(query, inverted_index):
    query = query.strip()
    qtokens = query.split()
    boolean_query = query_parse(qtokens)
    
    result_stack = list()
    for ind, token in enumerate(boolean_query):
        if(token not in ["AND","OR","NOT"]):
            result = set(inverted_index[token])
        else:
            if(token in ["AND","OR"]):
                right_op = result_stack.pop()
                left_op = result_stack.pop()
                if(token == "AND"):
                    operation = set.intersection
                else:
                    operation = set.union
                result = operation(left_op, right_op)
            else:
                operand = result_stack.pop()
                complement_docids = inverted_index[boolean_query[ind-1]]
                result = list()
                for word in inverted_index:
                    result.extend([doc_id for doc_id in inverted_index[word] if doc_id not in complement_docids])
                result = set(result)
        result_stack.append(result)
    return result_stack.pop()

In [ ]:
doc_ids = sorted(boolean_query("bartender",inverted_index))
print("bartender: ",doc_ids,"\n")
doc_ids = sorted(boolean_query("moon",inverted_index))
print("moon: ",doc_ids,"\n")
doc_ids = sorted(boolean_query("bartender AND moon",inverted_index))
print("bartender AND moon: ",doc_ids,"\n")

bartender:  [0, 192, 231, 522, 1047, 1357, 1371, 1772, 2159, 2178, 2468, 2522, 2685, 2820, 3009, 3331, 3428, 3891, 4064, 4134, 4555, 4624, 4642, 5026, 5229, 5433, 5737, 5792, 6380, 6583, 6603, 6642, 6767, 6797, 6884, 7069, 7166, 7184, 7231, 7271, 7667, 7759, 8140, 8307, 8371, 8766, 8775, 8790, 8858, 8928, 9060, 9142, 9201, 9298, 9492, 9556, 9769, 9916, 9951, 10180, 10192, 10447, 10569, 10656, 10678, 10813, 10859, 10924, 11122, 11280, 11350, 11477, 11614, 11652, 11662, 11717, 11760, 11781, 11870, 11887, 12025, 12151, 12290, 12319, 12381, 12812, 13016, 13032, 13050, 13058, 13087, 13103, 13182, 13197, 13204, 13267, 13337, 13391, 13398, 13425, 13444, 13454, 13482, 13568, 13633, 13673, 13686, 13699, 13865, 13928, 14000, 14050, 14070, 14072, 14273, 14692, 14782, 14927, 14966, 15045, 15215, 15559, 15598, 15657, 15830, 15832, 15839, 15879, 15929, 16144, 16258, 16343, 16379, 16385, 16423, 16580, 16605, 16626, 16631, 16668, 16789, 16856, 16857, 16918, 16938, 17046, 17077, 17267, 17908, 20465, 20

## Wild Card queries

In [ ]:
def wildcard_query(query, inverted_index):
    pattern = "^" + query.replace("*", "\w*") + "$"
    query = query.replace('*', '.')
    result = []
    terms_matched = []
    for word in inverted_index:
        if(re.search(query,word)):
            result.extend(inverted_index[word])
            terms_matched.append(word)
    result = list(set(result)) #removing duplicates
    print("Terms matched:", terms_matched)
    return result

In [ ]:
res = wildcard_query("bart*nder",inverted_index)
res
#Same as bartender

Terms matched: ['bartender', 'bartenders']


[0,
 16385,
 12290,
 11781,
 23046,
 15879,
 522,
 12812,
 4624,
 7184,
 11280,
 16918,
 1047,
 13337,
 12319,
 4642,
 4134,
 16423,
 9769,
 13865,
 16938,
 26165,
 15929,
 10813,
 8766,
 7231,
 21569,
 26690,
 8775,
 7759,
 13391,
 14927,
 9298,
 8790,
 11350,
 13398,
 12381,
 11870,
 7271,
 13928,
 5737,
 10859,
 5229,
 2159,
 6767,
 11887,
 13425,
 8307,
 14966,
 2685,
 2178,
 13444,
 23687,
 6797,
 13454,
 17046,
 8858,
 21148,
 5792,
 20646,
 13482,
 17066,
 10924,
 20652,
 14000,
 8371,
 17077,
 9916,
 192,
 16580,
 15045,
 15559,
 16075,
 10447,
 11477,
 13016,
 22233,
 16605,
 9951,
 8928,
 14050,
 6884,
 231,
 13032,
 1772,
 6380,
 15598,
 34541,
 16626,
 20723,
 14070,
 16631,
 14072,
 12025,
 13050,
 13568,
 13058,
 3331,
 2820,
 21766,
 16144,
 9492,
 16668,
 13087,
 15657,
 13103,
 3891,
 34099,
 5433,
 22330,
 13633,
 10569,
 1357,
 9556,
 1371,
 11614,
 21346,
 3428,
 9060,
 14692,
 13673,
 15215,
 11122,
 17267,
 13686,
 12151,
 13182,
 16258,
 13699,
 11652,
 21378,
 2

In [ ]:
res = wildcard_query("martyr*",inverted_index)
res

Terms matched: ['martyrs', 'martyred', 'martyrdom', 'martyring', 'martyre']


[32518,
 17031,
 10894,
 30615,
 18719,
 26528,
 14373,
 26414,
 6191,
 19118,
 6073,
 11194,
 27067,
 21949,
 17989,
 26181,
 32968,
 26953,
 25806,
 29903,
 22621,
 10974,
 5475,
 22633,
 19822,
 28654,
 22639,
 15603,
 28279,
 21626,
 16383]

In [ ]:
res = wildcard_query("*omination",inverted_index)
res

Terms matched: ['nomination', 'domination', 'denomination', 'abomination', 'abominations', 'nominations', 'denominations', 'denominational']


[30727,
 7688,
 8719,
 12305,
 13331,
 23574,
 17944,
 22049,
 26661,
 8238,
 15414,
 3131,
 28740,
 8773,
 30788,
 14921,
 11871,
 26212,
 15460,
 16491,
 8822,
 16506,
 32894,
 30854,
 21126,
 7818,
 32906,
 20132,
 2214,
 33959,
 4778,
 32941,
 17071,
 22703,
 16051,
 31420,
 17113,
 13530,
 31963,
 16092,
 13021,
 31986,
 15097,
 4345,
 25340,
 1276,
 3856,
 19219,
 9492,
 7446,
 32536,
 20761,
 9498,
 4395,
 7982,
 14639,
 7474,
 7479,
 17720,
 32055,
 20284,
 13632,
 2369,
 10562,
 19780,
 2891,
 13644,
 33621,
 22367,
 33634,
 32614,
 9067,
 21867,
 20843,
 8051,
 23416,
 32633,
 1916,
 12678,
 33162,
 7570,
 13716,
 20374,
 30616,
 7064,
 26014,
 5022,
 420,
 20401,
 6578,
 9651,
 12725,
 8119,
 8126,
 5063,
 7623,
 19912,
 7115,
 7633,
 7634,
 12244,
 34262,
 5085,
 12259,
 16359,
 25576,
 21999,
 10227]

In [ ]:
def search(query):
    if '*' in query:
        query = query.replace('*', '')
        result = []
        for word in inverted_index:
            if query in word:
                result.extend(inverted_index[word])
        return result
    elif '"' in query:
        query = query.replace('"', '')
        result = []
        for word in inverted_index:
            if query == word:
                result.extend(inverted_index[word])
        return result
    else:
        return inverted_index[query]

In [ ]:
df['id'] = range(0, len(df))


In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Retrieving relevant text using similarity index

In [ ]:
# similarity index
def retrieve_relevantText(query,inverted_index):
    Doc = wildcard_query(query,inverted_index)
    Text_Relevant = []
    for doc in Doc:
        Text_Relevant.append(df.loc[df['id'] == doc]['Plot'].values[0])
    return Text_Relevant

In [ ]:
retrieve_relevantText('bartender',inverted_index)


Terms matched: ['bartender', 'bartenders']


["a bartender is working at a saloon, serving drinks to customers. after he fills a stereotypically irish man's bucket with beer, carrie nation and her followers burst inside. they assault the irish man, pulling his hat over his eyes and then dumping the beer over his head. the group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. the bartender then sprays seltzer water in nation's face before a group of policemen appear and order everybody to leave.[1]",
 "in 1984, middle school student donny berger is in detention and begins a sexual relationship with his teacher, mary mcgarricle. when this relationship is discovered, she is sentenced to a maximum penalty prison term of 30 years, where she is revealed pregnant. custody of the unborn child is given to donny's abusive father until donny turns 18 to assume full custody.\r\nin 2012, donny is a broke, alcoholic slacker who spends his time with his friends, local drunk kenny, bartender brie and 

## LEMMATIZATION

In [ ]:
final_df_lemma_word = []
wordnet_lemmatizer = WordNetLemmatizer()
for line in df['Plot']:
  lemma_word=[];
  for w in line:
      word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
      word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
      word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
      lemma_word.append(word1)
  lemma_word= [word for word in lemma_word if word.isalnum()]
  final_df_lemma_word.append(lemma_word)

print(final_df_lemma_word[0:5])

[['a', 'b', 'a', 'r', 't', 'e', 'n', 'd', 'e', 'r', 'i', 's', 'w', 'o', 'r', 'k', 'i', 'n', 'g', 'a', 't', 'a', 's', 'a', 'l', 'o', 'o', 'n', 's', 'e', 'r', 'v', 'i', 'n', 'g', 'd', 'r', 'i', 'n', 'k', 's', 't', 'o', 'c', 'u', 's', 't', 'o', 'm', 'e', 'r', 's', 'a', 'f', 't', 'e', 'r', 'h', 'e', 'f', 'i', 'l', 'l', 's', 'a', 's', 't', 'e', 'r', 'e', 'o', 't', 'y', 'p', 'i', 'c', 'a', 'l', 'l', 'y', 'i', 'r', 'i', 's', 'h', 'm', 'a', 'n', 's', 'b', 'u', 'c', 'k', 'e', 't', 'w', 'i', 't', 'h', 'b', 'e', 'e', 'r', 'c', 'a', 'r', 'r', 'i', 'e', 'n', 'a', 't', 'i', 'o', 'n', 'a', 'n', 'd', 'h', 'e', 'r', 'f', 'o', 'l', 'l', 'o', 'w', 'e', 'r', 's', 'b', 'u', 'r', 's', 't', 'i', 'n', 's', 'i', 'd', 'e', 't', 'h', 'e', 'y', 'a', 's', 's', 'a', 'u', 'l', 't', 't', 'h', 'e', 'i', 'r', 'i', 's', 'h', 'm', 'a', 'n', 'p', 'u', 'l', 'l', 'i', 'n', 'g', 'h', 'i', 's', 'h', 'a', 't', 'o', 'v', 'e', 'r', 'h', 'i', 's', 'e', 'y', 'e', 's', 'a', 'n', 'd', 't', 'h', 'e', 'n', 'd', 'u', 'm', 'p', 'i', 'n'

In [ ]:
df['lemmatized_words']=final_df_lemma_word
df.head()

,Title,Plot,tokens,id,lemmatized_words
0,Kansas Saloon Smashers,"a bartender is working at a saloon, serving dr...","[bartender, working, saloon, serving, drinks, ...",0,"[a, b, a, r, t, e, n, d, e, r, i, s, w, o, r, ..."
1,Love by the Light of the Moon,"the moon, painted with a smiling face hangs ov...","[moon, painted, smiling, face, hangs, park, ni...",1,"[t, h, e, m, o, o, n, p, a, i, n, t, e, d, w, ..."
2,The Martyred Presidents,"the film, just over a minute long, is composed...","[film, minute, long, composed, two, shots, fir...",2,"[t, h, e, f, i, l, m, j, u, s, t, o, v, e, r, ..."
3,"Terrible Teddy, the Grizzly King",lasting just 61 seconds and consisting of two ...,"[lasting, 61, seconds, consisting, two, shots,...",3,"[l, a, s, t, i, n, g, j, u, s, t, 6, 1, s, e, ..."
4,Jack and the Beanstalk,the earliest known adaptation of the classic f...,"[earliest, known, adaptation, classic, fairyta...",4,"[t, h, e, e, a, r, l, i, e, s, t, k, n, o, w, ..."


# Retrieve relevant text using liklelihood language model

In [ ]:
text_list = df['lemmatized_words'].sum()


def generate_ngrams(text, n):
    n_grams = ngrams(text, n)
    return [' '.join(grams) for grams in n_grams]


all_ngrams = []
for i in range(1, 4):
    all_ngrams += generate_ngrams(text_list, i)

query = "bartender"

def calculate_likelihood(query, n):
    all_ngrams = []
    for line in df['lemmatized_words']:
        ngrams = nltk.ngrams(line, n)
        all_ngrams.extend(ngrams)
    query_ngrams = list(nltk.ngrams(query.split(), n))
    query_count = len(query_ngrams)
    match_count = sum([1 for ngram in query_ngrams if ngram in all_ngrams])
    if query_count == 0:
        return 0
    else:
        return match_count / query_count

scores = []
for n in range(1, 4):
    score = calculate_likelihood(query, n)
    scores.append(score)
    print(f"Likelihood score for {n}-gram model: {score}")

## Ranking of retrieved documents

In [ ]:
def rank_retrieved_documents(query,inverted_index):
    Doc = wildcard_query(query,inverted_index)
    Text_Relevant = []
    for doc in Doc:
        Text_Relevant.append(df.loc[df['id'] == doc]['Plot'].values[0])
    Score = {}
    for doc in Text_Relevant:
        Score[doc] = 0
        for word in query.split():
            Score[doc] += doc.count(word)
    Score_Sort = sorted(Score.items(), key=lambda x: x[1], reverse=True)
    return Score_Sort

In [ ]:
rank_retrieved_documents('bartender',inverted_index)

Terms matched: ['bartender', 'bartenders']


[("two slave brothers, african-american levi (anthony scott) and albino sunshine (chris robinson), discover a treasure map while digging on the plantation of their master striker (ted cassidy). though striker initially takes the map from them, they take it back and escape, determined to find the treasure in order to set themselves free. striker pursues them, eventually stopping at a seedy bar to recruit helpers who will aid him in his pursuit; he recruits a mysterious bounty hunter, as well as the bartender (robert leslie). the two slaves follow the map continuously through the woods and swamps, regularly confronting and escaping various dangers from snakes to a group of rednecks. they briefly find refuge in an escaped slaves' camp, but striker and his men follow the trail straight to the camp and run out the population. striker is noticeably disturbed at the bartender for killing a woman and an old man, and briefly pulls his own gun on the man.\r\nthe brothers continue on further and 

## Advanced search

In [ ]:
def advanced_search(query,inverted_index):
    Text_Relevant = retrieve_relevantText(query,inverted_index)
    Score = {}
    for doc in Text_Relevant:
        Score[doc] = 0
        for word in query.split():
            Score[doc] += doc.count(word)
    Score_Sort = sorted(Score.items(), key=lambda x: x[1], reverse=True)
    return Score_Sort


In [ ]:
advanced_search('bartender',inverted_index)

Terms matched: ['bartender', 'bartenders']


[("two slave brothers, african-american levi (anthony scott) and albino sunshine (chris robinson), discover a treasure map while digging on the plantation of their master striker (ted cassidy). though striker initially takes the map from them, they take it back and escape, determined to find the treasure in order to set themselves free. striker pursues them, eventually stopping at a seedy bar to recruit helpers who will aid him in his pursuit; he recruits a mysterious bounty hunter, as well as the bartender (robert leslie). the two slaves follow the map continuously through the woods and swamps, regularly confronting and escaping various dangers from snakes to a group of rednecks. they briefly find refuge in an escaped slaves' camp, but striker and his men follow the trail straight to the camp and run out the population. striker is noticeably disturbed at the bartender for killing a woman and an old man, and briefly pulls his own gun on the man.\r\nthe brothers continue on further and 

## Re-Rank

In [ ]:
def rerank_results(query,inverted_index):
    Idx = inverted_index
    Doc = wildcard_query(query,inverted_index)
    Score = []
    query_terms = query.split()
    for doc_id in Doc:
        score = 0
        for term in query_terms:
            if term in Idx and doc_id in Idx[term]:
                try:
                    score += Idx[term][doc_id]
                except:
                    pass
        Score.append((doc_id, score))
    Score.sort(key=lambda x: x[1], reverse=True)
    return [doc_id for doc_id, score in Score]

In [ ]:
rerank_results('bartender',inverted_index)

Terms matched: ['bartender', 'bartenders']


[0,
 16385,
 12290,
 11781,
 23046,
 15879,
 522,
 12812,
 4624,
 7184,
 11280,
 16918,
 1047,
 13337,
 12319,
 4642,
 4134,
 16423,
 9769,
 13865,
 16938,
 26165,
 15929,
 10813,
 8766,
 7231,
 21569,
 26690,
 8775,
 7759,
 13391,
 14927,
 9298,
 8790,
 11350,
 13398,
 12381,
 11870,
 7271,
 13928,
 5737,
 10859,
 5229,
 2159,
 6767,
 11887,
 13425,
 8307,
 14966,
 2685,
 2178,
 13444,
 23687,
 6797,
 13454,
 17046,
 8858,
 21148,
 5792,
 20646,
 13482,
 17066,
 10924,
 20652,
 14000,
 8371,
 17077,
 9916,
 192,
 16580,
 15045,
 15559,
 16075,
 10447,
 11477,
 13016,
 22233,
 16605,
 9951,
 8928,
 14050,
 6884,
 231,
 13032,
 1772,
 6380,
 15598,
 34541,
 16626,
 20723,
 14070,
 16631,
 14072,
 12025,
 13050,
 13568,
 13058,
 3331,
 2820,
 21766,
 16144,
 9492,
 16668,
 13087,
 15657,
 13103,
 3891,
 34099,
 5433,
 22330,
 13633,
 10569,
 1357,
 9556,
 1371,
 11614,
 21346,
 3428,
 9060,
 14692,
 13673,
 15215,
 11122,
 17267,
 13686,
 12151,
 13182,
 16258,
 13699,
 11652,
 21378,
 2

## Finding out query intention

In [ ]:
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def query_intention(query):
    
    words = nltk.word_tokenize(query)
    
    tagged_words = nltk.pos_tag(words)
    
    tag_counts = nltk.Counter(tag for word, tag in tagged_words)
    most_common_tag = tag_counts.most_common(1)[0][0]
    
    if most_common_tag in ['NN', 'NNS', 'NNP', 'NNPS']:
        return 'Informational'
    elif most_common_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return 'Transactional'
    else:
        return 'Other'


In [ ]:
query1 = "He misses watching movies, and acting"
print(query_intention(query1))

Transactional


In [ ]:
query2 = "the film starts on ocean drive in miami"
print(query_intention(query2)) 

Informational


In [ ]:
query3 = "His love is unconditional"
print(query_intention(query3))  

Other


## Relevance feedback

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['lemmatized_words'].apply(lambda x: ' '.join(x)))

def get_top_similar(query, n=5):
    query_tfidf = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    top_indices = cosine_similarities.argsort()[::-1][:n]
    ranking_scores = cosine_similarities[top_indices]
    top_docs = df.iloc[top_indices][['Plot', 'Title', 'lemmatized_words']]
    top_docs['ranking_score'] = ranking_scores
    return top_docs

# initial query
query = "bartender"
print('\n')

top_similar_docs = get_top_similar(query)

print('Top 5 most similar documents to the query:')
print(top_similar_docs[['Plot', 'Title', 'ranking_score']])

#checking if it is relevent or irrelevent
feedback = []
for i in range(len(top_similar_docs)):
    doc_feedback = ('Document {1}:irrelevent','Document {2}:irrelevent','Document {3}:relevent','Document {4}:relevent','Document {5}:relevent')
    feedback.append(doc_feedback)

for i in range(len(feedback)):
    if feedback[i] == 'relevant':
        relevant_words = top_similar_docs.iloc[i]['lemmatized_words']
        query += ' ' + ' '.join(relevant_words)
    elif feedback[i] == 'irrelevant':
        irrelevant_words = top_similar_docs.iloc[i]['lemmatized_words']
        query_words = query.split()
        query_words = [w for w in query_words if w not in irrelevant_words]
        query = ' '.join(query_words)

top_similar_docs = get_top_similar(query)
print('\nNew top 5 most similar documents to the updated query:')
print(top_similar_docs[['Plot', 'Title', 'ranking_score']])

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(




Top 5 most similar documents to the query:
                                                    Plot  \
6210                      "a beach near rio de janeiro".   
18182  a respected judge leads a double life as a mur...   
18620  two young teachers travel to the yorkshire moo...   
17461  len (bryan brown) and his wife barbara (kris m...   
27172  a story of rebirth and revenge in the midst of...   

                                    Title  ranking_score  
6210                   They Were So Young       0.273132  
18182                  Condemned to Death       0.244052  
18620                  The Night Has Eyes       0.242615  
17461  The Love Letters from Teralba Road       0.242098  
27172                     Bangarada Jinke       0.241272  

New top 5 most similar documents to the updated query:
                                                    Plot  \
6210                      "a beach near rio de janeiro".   
18182  a respected judge leads a double life as a mur...   
1862

## Semantic Matching 

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize



def calculate_similarity(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    if not synsets1 or not synsets2:
        return 0.0
    max_sim = -1
    for synset1 in synsets1:
        for synset2 in synsets2:
            sim = wn.path_similarity(synset1, synset2)
            if sim is not None and sim > max_sim:
                max_sim = sim
    return max_sim



def semantic_matching(query):
    documents = inverted_index

    Score = []

    Qry_Token = word_tokenize(query)

    for Doc in documents:
        doc_tokens = word_tokenize(Doc)
        similarity_score = 0.0
        for query_token in Qry_Token:
            max_sim = -1
            for doc_token in doc_tokens:
                sim = calculate_similarity(query_token, doc_token)
                if sim > max_sim:
                    max_sim = sim
            similarity_score += max_sim
        Score.append((Doc, similarity_score / len(Qry_Token)))
    return Score

In [ ]:
semantic_matching('bartender')

[('bartender', 1.0),
 ('working', 0.1),
 ('saloon', 0.08333333333333333),
 ('serving', 0.09090909090909091),
 ('drinks', 0.1111111111111111),
 ('customers', 0.14285714285714285),
 ('fills', 0.1),
 ('stereotypically', 0.1),
 ('irish', 0.1),
 ('man', 0.2),
 ('bucket', 0.08333333333333333),
 ('beer', 0.09090909090909091),
 ('carrie', 0.0),
 ('nation', 0.125),
 ('followers', 0.2),
 ('burst', 0.1),
 ('inside', 0.1),
 ('assault', 0.1),
 ('pulling', 0.09090909090909091),
 ('hat', 0.08333333333333333),
 ('eyes', 0.09090909090909091),
 ('dumping', 0.09090909090909091),
 ('head', 0.16666666666666666),
 ('group', 0.1111111111111111),
 ('begin', 0.125),
 ('wrecking', 0.09090909090909091),
 ('bar', 0.1),
 ('smashing', 0.1),
 ('fixtures', 0.1111111111111111),
 ('mirrors', 0.09090909090909091),
 ('breaking', 0.1),
 ('cash', 0.1111111111111111),
 ('register', 0.1),
 ('sprays', 0.09090909090909091),
 ('seltzer', 0.08333333333333333),
 ('water', 0.125),
 ('face', 0.2),
 ('policemen', 0.125),
 ('appear',

**Implementation of positional indexing**


In [ ]:
def construct_positional_posting_list(tokenized_corpus):
  positional_index = dict()
  for id, plot in enumerate(tokenized_corpus):
    for token_id, token in enumerate(plot):
      if token not in positional_index:
        positional_index[token] = dict()
      if id not in positional_index[token]:
        positional_index[token][id] = list()
      positional_index[token][id].append(token_id)
  for token in positional_index:
    for id in positional_index[token]:
      positional_index[token][id] = sorted(positional_index[token][id])
    items=list(positional_index[token].items())
    items.sort(key = lambda x:x[0])
    for k,v in items:
      positional_index[token][k]=v
  return positional_index
  

In [ ]:
positional_index = construct_positional_posting_list(df["tokens"])

In [ ]:
print("bartender:", positional_index["bartender"])
print("moon:", positional_index["bartender"])

bartender: {0: [1, 65], 192: [42], 231: [115, 181, 188, 216], 522: [99], 1047: [242, 266], 1357: [47], 1371: [372], 1772: [32], 2159: [105], 2178: [268], 2468: [426], 2522: [194], 2685: [94, 124, 132], 2820: [6], 3009: [185], 3331: [424], 3428: [199], 3891: [112, 161], 4064: [820], 4134: [367], 4555: [314], 4624: [93], 4642: [725], 5026: [43], 5229: [142, 262], 5433: [382], 5737: [473], 5792: [94], 6380: [116, 138], 6583: [98], 6603: [329], 6642: [27], 6767: [38], 6797: [120], 6884: [15], 7069: [139], 7166: [384], 7184: [73], 7231: [126], 7271: [537], 7667: [366], 7759: [127], 8140: [31, 839, 859], 8307: [57], 8371: [968, 992], 8766: [82, 142, 275, 303], 8775: [15], 8790: [366], 8858: [162], 8928: [119], 9060: [592], 9142: [198], 9201: [117], 9298: [580, 670, 684, 704], 9492: [215], 9556: [357], 9769: [363], 9916: [314], 9951: [331, 352], 10180: [56], 10192: [138], 10447: [301], 10569: [1013], 10656: [93, 177], 10678: [124, 143], 10813: [486], 10859: [21], 10924: [9, 168, 385], 11122: 

In [ ]:
def positional_intersection(p, q, diff):
  ret = list()
  i = 0
  j = 0
  while i < len(p) and j < len(q):
    p_docid = list(p.keys())[i]
    q_docid = list(q.keys())[j]
    if p_docid == q_docid:
      l = list()
      pp = p[p_docid]
      qq = q[q_docid]
      k = 0
      while(k<len(pp)):
        m = 0
        while(m<len(qq)):
          gap = qq[m]-pp[k]
          #print(qq[m],pp[k],gap,'\n')
          if gap == diff:
            l.append(m)
          m+=1
        for ps in l:
          gap = qq[ps]-pp[k]
          if gap != diff:
            l.remove(ps)
        for ps in l:
          ret.append((p_docid,pp[k],qq[ps]))
        k+=1
      i+=1
      j+=1
    elif p_docid<q_docid:
      i+=1
    else:
      j+=1
  return ret

In [ ]:
bartender = {
    1: [1,3],
    2: [3,5]
}
moon = {
    1:[2],
    2:[4]
}

In [ ]:
positional_intersection(bartender,moon,1)

[(1, 1, 2), (2, 3, 4)]

In [ ]:
def search_positional_index(query):
  query = query.split()
  words = [query[i] for i in range(0,len(query),2)]
  k = [int(query[i][1:]) for i in range(1,len(query),2)]
  doc_list = list()
  for i in range(0,len(words)-1):
    word1,word2 = words[i:i+2]
    p = positional_index[word1]
    q = positional_index[word2]
    #print(p,'\n',q,'\n')
    result = positional_intersection(p,q,k[i])
    doc_list.extend(result)
  return doc_list


In [ ]:
search_positional_index("bartender /1 working")

[(0, 0, 1), (10180, 33, 34)]

In [ ]:
search_positional_index("moon /2 smiling")

[(1, 0, 2)]

**The end**